# Building an image retrieval system with deep features


## 1. Fire up GraphLab Create

In [1]:
import graphlab
graphlab.canvas.set_target('ipynb')

## 2. Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1542048530.log
INFO:graphlab.cython.cy_server:GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1542048530.log


This non-commercial license of GraphLab Create for academic use is assigned to bernardo.ronquillo@gmail.com and will expire on May 11, 2019.


## 3. Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

In [5]:
image_train[0]

{'deep_features': array('d', [0.24287176132202148, 1.0954537391662598, 0.0, 0.39362990856170654, 0.0, 0.0, 11.894915580749512, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5788496136665344, 0.4954667389392853, 2.5141289234161377, 0.0, 1.5180106163024902, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5813961029052734, 0.0, 0.0, 2.595609426498413, 2.7079553604125977, 0.0, 0.0, 0.0, 0.8509902954101562, 0.0, 0.7203489542007446, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2700355052947998, 0.0, 0.0, 0.0, 0.0, 0.08592796325683594, 0.0, 0.7010231018066406, 0.0, 0.0, 0.0, 0.0, 0.024805665016174316, 0.0, 0.0, 0.17549043893814087, 0.0, 0.0, 0.0, 0.0, 0.0, 2.392784595489502, 0.0, 0.0, 4.471865653991699, 0.0, 1.6358323097229004, 0.0, 4.417484760284424, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4117904901504517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1247677206993103, 0.0, 0.0, 0.8957164287567139, 0.0, 0.0, 0.3334987759590149, 0.0, 0.0, 0.20787304639816284, 2.

In [6]:
print image_train.shape,"\n"
image_train['image'].show()

(2005, 5) 



### Instances of CATS

In [7]:
cats = image_train.filter_by('cat', 'label')
print cats.shape,"\n"
print cats['id']
print cats['image'].show()

(509, 5) 

[33, 36, 159, 331, 367, 384, 494, 597, 788, 882, 1252, 1696, 1778, 1785, 1846, 1969, 2057, 2068, 2082, 2127, 2168, 2386, 2501, 2564, 2668, 2699, 2738, 3026, 3043, 3052, 3105, 3461, 3514, 3756, 3785, 3792, 3863, 3875, 3956, 4007, 4149, 4243, 4392, 4428, 4472, 4531, 4772, 4836, 4847, 4915, 4934, 5023, 5072, 5097, 5147, 5192, 5319, 5473, 5545, 5553, 5601, 5725, 5876, 5877, 5915, 6151, 6186, 6304, 6330, 6332, 6458, 6462, 6507, 6557, 6567, 6584, 6868, 6910, 6980, 6994, 7050, 7086, 7123, 7205, 7493, 7603, 7689, 7765, 7803, 7839, 8000, 8104, 8183, 8210, 8251, 8302, 8349, 8381, 8420, 8599, ... ]


None


### Instances of BIRDS

In [8]:
birds = image_train.filter_by('bird', 'label')
print birds.shape,"\n"
print birds['id']
print birds['image'].show()

(478, 5) 

[24, 90, 121, 138, 335, 560, 649, 775, 802, 975, 1129, 1209, 1440, 1444, 1523, 1641, 1693, 1789, 1798, 1812, 1852, 1921, 2091, 2163, 2177, 2372, 2408, 2535, 2869, 3012, 3281, 3402, 3517, 3608, 3626, 3629, 3708, 3869, 4017, 4154, 4242, 4293, 4367, 4376, 4549, 4604, 4608, 4830, 4895, 5219, 5226, 5248, 5296, 5527, 5574, 5715, 5724, 5892, 5933, 6019, 6023, 6041, 6230, 6341, 6392, 6477, 6802, 6805, 6944, 6963, 7037, 7068, 7098, 7135, 7216, 7258, 7340, 7452, 7466, 7480, 7525, 7951, 8055, 8121, 8166, 8195, 8234, 8438, 8477, 8641, 8736, 9016, 9089, 9172, 9215, 9227, 9408, 9420, 9448, 9523, ... ]


None


### Instances of AUTOMOBILES

In [9]:
automobiles = image_train.filter_by('automobile', 'label')
print automobiles.shape,"\n"
print automobiles['id']
automobiles['image'].show()

(509, 5) 

[97, 136, 302, 312, 323, 536, 593, 962, 997, 1421, 1448, 1494, 1548, 1604, 1736, 1869, 1907, 2046, 2249, 2268, 2362, 2422, 2496, 2568, 2570, 2727, 2862, 2928, 3020, 3131, 3230, 3231, 3327, 3414, 3523, 3779, 3954, 4006, 4013, 4084, 4458, 4609, 4631, 4664, 4769, 4932, 5070, 5137, 5197, 5202, 5243, 5257, 5287, 5392, 5590, 5709, 5769, 5939, 6241, 6244, 6318, 6623, 6659, 6678, 6769, 6851, 6871, 6885, 6928, 6971, 7019, 7148, 7271, 7350, 7438, 7513, 7579, 7707, 7862, 8090, 8093, 8149, 8232, 8244, 8540, 8735, 8801, 8855, 8920, 8938, 8977, 9056, 9118, 9216, 9232, 9327, 9565, 9781, 9997, 10153, ... ]


### Instances of DOGS

In [38]:
dogs = image_train.filter_by('dog', 'label')
print dogs.shape,"\n"
print dogs['id']
dogs['image'].show()

(509, 5) 

[70, 107, 177, 424, 462, 542, 573, 851, 919, 1172, 1206, 1298, 1401, 1411, 2117, 2167, 2194, 2305, 2410, 2634, 2688, 2730, 2736, 2788, 2843, 2940, 2984, 3008, 3021, 3040, 3082, 3167, 3187, 3264, 3270, 3367, 3410, 3420, 3431, 3616, 3767, 3803, 3828, 3896, 3949, 3993, 4125, 4362, 4366, 4495, 4600, 4610, 4692, 4884, 4922, 5039, 5267, 5369, 5379, 5467, 5606, 5695, 5755, 5830, 5850, 5960, 6037, 6094, 6104, 6131, 6184, 6580, 6642, 6675, 6874, 7031, 7105, 7685, 7773, 7776, 7901, 8040, 8252, 8265, 8274, 8282, 8463, 8500, 8684, 8851, 9286, 9450, 9871, 9999, 10181, 10223, 10270, 10315, 10397, 10521, ... ]


## 4. Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [13]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

## 5. Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

We are going to create a simple function to view the nearest neighbors to save typing:

In [14]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [27]:
cat = cats[12:13]
cat['image'].show()

In [32]:
get_images_from_ids(knn_model.query(cat))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.598ms      |

| Done         |         | 100         | 70.395ms     |

+--------------+---------+-------------+--------------+

### Finding similar images to a car

In [22]:
car = automobiles[99:100]
car['image'].show()

In [23]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.817ms      |

| Done         |         | 100         | 68.066ms     |

+--------------+---------+-------------+--------------+

### Finding similar images to a bird

In [34]:
bird = birds[7:8]
bird['image'].show()

In [35]:
get_images_from_ids(knn_model.query(bird))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.495ms      |

| Done         |         | 100         | 69.601ms     |

+--------------+---------+-------------+--------------+

### Finding similar images to a dog

In [40]:
dog = dogs[1:2]
dog['image'].show()

In [41]:
get_images_from_ids(knn_model.query(dog))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.067ms      |

| Done         |         | 100         | 71.125ms     |

+--------------+---------+-------------+--------------+

## 6. Lambdas in Python: Find neighbours with just a line
let's create a lambda to find and show nearest neighbor images

In [43]:
show_neighbor_cats = lambda i: get_images_from_ids(knn_model.query(cats[i:i+1]))['image'].show()
show_neighbor_birds = lambda i: get_images_from_ids(knn_model.query(birds[i:i+1]))['image'].show()
show_neighbor_automobiles = lambda i: get_images_from_ids(knn_model.query(automobiles[i:i+1]))['image'].show()
show_neighbor_dogs = lambda i: get_images_from_ids(knn_model.query(dogs[i:i+1]))['image'].show()

In [45]:
show_neighbor_cats(12)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.581ms      |

| Done         |         | 100         | 70.704ms     |

+--------------+---------+-------------+--------------+

In [48]:
show_neighbor_birds(7)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.724ms      |

| Done         |         | 100         | 67.489ms     |

+--------------+---------+-------------+--------------+

In [49]:
show_neighbor_dogs(1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.433ms      |

| Done         |         | 100         | 69.939ms     |

+--------------+---------+-------------+--------------+

## 7. Evaluate with the test set

In [50]:
image_test = graphlab.SFrame('image_test_data/')

In [70]:
image_test.shape

(4000, 5)

In [75]:
cats_test = image_test.filter_by('cat', 'label')
birds_test = image_test.filter_by('bird', 'label')
automobiles_test = image_test.filter_by('automobile', 'label')
dogs_test = image_test.filter_by('dog', 'label')

In [81]:
items = 10
cat = cats_test[0:items]
cat['image'].show()

bird = birds_test[0:items]
bird['image'].show()

automobile = automobiles_test[0:items]
automobile['image'].show()

dog = dogs_test[0:items]
dog['image'].show()

In [82]:
show_neighbor_cats_test = lambda i: get_images_from_ids(knn_model.query(cats_test[i:i+1]))['image'].show()
show_neighbor_birds_test = lambda i: get_images_from_ids(knn_model.query(birds_test[i:i+1]))['image'].show()
show_neighbor_automobiles_test = lambda i: get_images_from_ids(knn_model.query(automobiles_test[i:i+1]))['image'].show()
show_neighbor_dogs_test = lambda i: get_images_from_ids(knn_model.query(dogs_test[i:i+1]))['image'].show()

In [84]:
show_neighbor_cats_test(0)
show_neighbor_birds_test(0)
show_neighbor_automobiles_test(0)
show_neighbor_dogs_test(0)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.967ms      |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 6.525ms      |

| Done         |         | 100         | 67.385ms     |

+--------------+---------+-------------+--------------+

In [88]:
show_neighbor_birds_test(200)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 7.366ms      |

| Done         |         | 100         | 70.535ms     |

+--------------+---------+-------------+--------------+